# Classify global climate scenarios using the Koppen Geiger Climate Classifications

## Import neccessary packages and authenticate earth engine

In [7]:
import ee
import pandas as pd
import matplotlib.pyplot as plt

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-koppengeiger')

## Import data

In [8]:
## Input Data
# Import the NASA-GDDP CMIP6 dataset for ensemble of future climate scenarios 
GDDP = ee.ImageCollection("NASA/GDDP-CMIP6")

# Import the GTOPO 30 arc second Digital Elevation Model (DEM) for elevation data, e.g. for calculating latitudinal bio temperature
demOrig = ee.Image("USGS/GTOPO30")
demReprojected = demOrig.reproject(GDDP.first().projection()) ##Reproject the DEM to GDDP crs
lapseRateScalar = -6 #degC/km 
lapseRateGlobal = demReprojected.divide(1000).multiply(lapseRateScalar) #bioTemperatre adjustments for sea level temperature using avg lapse rate

## Analysis Data
# Import World region outlines for reducing regional statistics
table = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")

In [6]:
#-----------------------Data imports--------------------------          
imageCollection = ee.ImageCollection("NASA/GDDP-CMIP6").select(['tas','pr']),
nexdata = ee.Image("NASA/NEX-GDDP/historical_ACCESS1-0_19500105"),
dem = ee.Image("USGS/GTOPO30"),
climClass = ee.Image("users/climateClass/world_koppen"),
climClassVis = {"opacity":1,"bands":["b1"],"min":1,"max":32,"gamma":1},
kopVisParam = {"opacity":1,"bands":["constant"],"min":1,"max":30,"palette":["0000ff","0078ff","46aafa","ff0000","ff9696","ffa929","ffdc64","ffff00","c8c800","969600","96ff96","64c864","329632","c8ff50","64ff32","32c800","ff00ff","c800c8","963296","966496","aaafff","5a78dc","4b50b4","320087","00ffff","32c8ff","007d7d","00465f","b3b3b3","666666"]},
landimg = ee.Image("Oxford/MAP/LST_Night_5km_Monthly/2005_10");
land=landimg.select('Mean').gt(-999);

## Functions to process the data

## Algorithm to classify Koppen Geiger climate classes

In [11]:

  
#-------------------------Code to classify Koppen-Geiger Climate Classess------------------------    

def KOPPEN(year, year2, rcp):
    #d = ee.Date.fromYMD(year,1,1) #day of the year iterator
    #Ensamble = [];                #empty list to store the Climate model ensamble
    
    start = ee.Date.fromYMD(year,1,1)
    end = ee.Date.fromYMD(year2,12,31)
    
    n_days = end.difference(start,'day')
    date_list = ee.List.sequence(0,n_days).map(function(n){
      return start.advance(n,'day')})
      
    Ensamble_means = ee.List([])
    if(year < 2016) {
      Ensamble_means = date_list.map(function(date) {
      img_daily = imageCollection.filterDate(date)
        .filter(ee.Filter.eq('scenario', 'historical'))
      e_mean =  img_daily.mean()
      date_dict = ee.Dictionary(['day', img_daily.first().get('day'),'month', img_daily.first().get('month'),'year', img_daily.first().get('year')]);
      return e_mean.set(date_dict)  
    })}
    
    if(year > 2015){
      Ensamble_means = date_list.map(function(date) {
      img_daily = imageCollection.filterDate(date)
        .filter(ee.Filter.eq('scenario', rcp))
        #.filter(ee.Filter.neq('model','BCC-CSM2-MR')) # there is an issue with air temperature for 'NorESM2-LM' (only for scenario 'ssp585' in 2096) need to calculate the average of 
      e_mean =  img_daily.mean()
      date_dict = ee.Dictionary(['day', img_daily.first().get('day'),'month', img_daily.first().get('month'),'year', img_daily.first().get('year')]);
      return e_mean.set(date_dict)  
    })}
    Ensamble_means = ee.ImageCollection(Ensamble_means)
    
    #Now we loop through the period and extract the ensamble mean for each date and store in our list 
    '''
    for (i = 0; i < n_days; i++) { 
      img_daily = imageCollection.filterDate(d).limit(21); #Get the RCP45 ensamble 
      d = d.advance(1,'day')                               #Add a day to the runner
      #Create dictionary to store the date info for the ensamble
      date_dict = ee.Dictionary(['day', img_daily.first().get('day'),'month', img_daily.first().get('month'),'year', img_daily.first().get('year')]);
      #Calculate the mean of the climate model ensamble and attach the date dictionary 
      ensamb_mean = ee.Image(img_daily.reduce(ee.Reducer.mean())).set(date_dict);
      Ensamble.push(ensamb_mean);                     #store the daily ensemble mean
    }
    
    Ensamble_means = ee.ImageCollection(Ensamble);#Classify as image collection to satisfy GEE
    '''

    #----------------------Köppen Geiger Classification Algorithm------------------------
    
    #Extract image.collections with our Köppen-Geiger input data
    tmax_year = Ensamble_means.select('tas')
    prec_year = Ensamble_means.select('pr')
    
    #need to find the average precip and tmax in each month
    #Start by creating empty lists and and defining the calender 
    prec_month = []; tmax_month = [];
    days_in_month = [31,28,31,30,31,30,31,31,30,31,30,31];
    
    for (i = 0; i < 12; i++) { 
      #extract the daily temp and precip
      prec = prec_year.filter(ee.Filter.eq('month',i+1));
      tma = tmax_year.filter(ee.Filter.eq('month',i+1));
      
      #Define the date and calculate the monthly mean temp and precip
      date_dict = ee.Dictionary(['month',prec.first().get('month'),'year',prec.first().get('year')]);
      prec_monthly_mean = ee.Image(prec.reduce(ee.Reducer.mean())).multiply(60*60*24*days_in_month[i]).set(date_dict);
      tmax_monthly_mean = ee.Image(tma.reduce(ee.Reducer.mean())).set(date_dict);
      
      tmax_month.push(tmax_monthly_mean);
      prec_month.push(prec_monthly_mean);
    }
    
    #define the lists as image collections
    prec_month = ee.ImageCollection(prec_month);
    tmax_month = ee.ImageCollection(tmax_month);
    
    #-----------------Calculating the Koppen Geiger Variables-------------------------
    #tmon10: Calculate the number of months that the temperature is above 10°C 
    tmon10 = ee.Image(0); #start by defining an empty image
    
    for (i = 0; i < 12; i++) { #loop over each month
    
    t_this_month = ee.Image(tmax_month.filter(ee.Filter.eq('month',i+1)).first()) ;  #extract each monthly temp image from the monthly averages
    img = ee.Image(0);                                                               #create an emtpy image to overlay the pixels with t > 10°C
    img = img.where(t_this_month.gte(283.15),1);                                         #Classify pixels with tmon > 10 °C
    tmon10 = tmon10.add(img);                                                            #Add up the months in the T10 image
    }
    
    #Summer/winter temperatures and precip
    ONDJFM = [10,11,12,1,2,3]; #Define Northern Hemisphere winter months
    AMJJAS = [4,5,6,7,8,9];    #Define Nothern Hemisphere summer months
    
    #Find NH summer and winter mean temperatures
    ONDJFM_t = ee.Image(tmax_year.filter(ee.Filter.inList('month',ONDJFM)).reduce(ee.Reducer.mean()));
    AMJJAS_t = ee.Image(tmax_year.filter(ee.Filter.inList('month', AMJJAS)).reduce(ee.Reducer.mean()));
    
    #Define summer and winter hemisphere based on whether the mean temp in ONDJFM is higher than AMJJAS 
    SH = ee.Image(0).where(ONDJFM_t.gt(AMJJAS_t),1);
    NH = ee.Image(0).where(AMJJAS_t.gte(ONDJFM_t),1);
    
    #Extract the summer and winter precip
    ONDJFM_prec = prec_month.filter(ee.Filter.inList('month', ONDJFM));
    AMJJAS_prec = prec_month.filter(ee.Filter.inList('month', AMJJAS));
    
    #MAP and MAT: Mean annual precip and temp
    MAP = ee.Image(prec_year.reduce(ee.Reducer.mean()).multiply(60*60*24*365));
    MAT = tmax_year.reduce(ee.Reducer.mean());
    
    #P_thresh: Threshold precipitation
    if((MAP.multiply(0.7)).gte(ONDJFM_prec)){   #70% of MAP occurs in winter
      P_thresh = ee.Image(MAT.subtract(273.15)).multiply(2);
    }else if((MAP.multiply(0.7)).gte(AMJJAS_prec)){   #70% of MAP occurs in summer
      P_thresh = ee.Image((MAT.subtract(273.15)).multiply(2)).add(28);
    }else{
      P_thresh = ee.Image((MAT.subtract(273.15)).multiply(2)).add(14);
    }  
    
    #MAP100
    MAP100 = ee.Image(100).subtract(ee.Image(MAP.divide(25)))
    
    #Create an empty image to store our Köppen classifications
    koppen = ee.Image(0)
    
    ###########/Numbering Convention################/
    ####/ Entire classes: A-100, B-200, C-300, D-400
    ####/ Subclasses: s---1, w---2, f---3
    ####/ Sub-Subclasses: as employed in Koppen-Geiger Classification Map
    ######################################
    #Class B: Arid
    koppen = koppen.where(MAP.lt(P_thresh.multiply(10)),
                    200);
    
    # BW: MAP<5×Pthreshold
    koppen = koppen.where(MAP.lt(P_thresh.multiply(5))
                    .and(koppen.eq(200)),
                    2001);
                    
    #BS: MAP>= 5xPthreshold              
    koppen = koppen.where(koppen.eq(200)
                    .and(MAP.gte(P_thresh.multiply(5))),
                    2002);
                  
            #BWh: MAT>=18
            koppen = koppen.where(MAT.gte(273.15+18)
                    .and(koppen.eq(2001)),
                    4);
              
            #BWk: MAT<18 
            koppen = koppen.where(MAT.lt(273.15+18)
                    .and(koppen.eq(2001)),
                    5);
                    
            #BSh: MAT>=18
            koppen = koppen.where(MAT.gte(273.15+18)
                    .and(koppen.eq(2002)),
                    6);
            
            #BSk: MAT<18
            koppen = koppen.where(MAT.lt(273.15+18) 
                    .and(koppen.eq(2002)), 
                    7); 
            
    #Class A: Tropical: Tcold >=18
    koppen = koppen.where(ee.Image(tmax_year.max()).gte(273.15+18) 
                    .and(koppen.neq(4))
                    .and(koppen.neq(5))
                    .and(koppen.neq(6))
                    .and(koppen.neq(7))
                    ,100)
                    
            #Af: Pdry >=60
            koppen = koppen.where(koppen.eq(100)
                            .and(ee.Image(prec_month.min()).gte(60)),
                            1);
    
            #Am: Not (Af) & Pdry>=100–MAP/25
            koppen = koppen.where(koppen.eq(100)
                           .and(koppen.neq(1))
                           .and(ee.Image(prec_month.min()).gte(MAP100)),
                           2);
    
            #Aw: Not (Af) & Pdry<100–MAP/25
            koppen = koppen.where(koppen.eq(100)
                           .and(koppen.neq(1))
                           .and(ee.Image(prec_month.min()).lt(MAP100)),
                           3)
    
    #Class C: Temperate 
    koppen = koppen.where(ee.Image(tmax_month.max()).gte(283.15)
                   .and(koppen.neq(4))
                   .and(koppen.neq(5))
                   .and(koppen.neq(6))
                   .and(koppen.neq(7))
                   .and(ee.Image(tmax_month.min()).gte(273.15))
                   .and(ee.Image(tmax_month.min()).lte(291.15)),
                   300)
    
    #Cs: Psdry<40 & Psdry<Pwwet/3
    PSHsdry = ee.Image(SH.multiply(ONDJFM_prec.min()));
    PNHsdry = ee.Image(NH.multiply(AMJJAS_prec.min()));
    PSHwwet = ee.Image(SH.multiply(AMJJAS_prec.max()));
    PNHwwet = ee.Image(NH.multiply(ONDJFM_prec.max()));
    
    koppen=koppen.where(PSHsdry.lt(40)
                 .or(PNHsdry.lt(40))
                 .and(PSHsdry.lt(PSHwwet.divide(3)))
                 .or(PNHsdry.lt(PNHwwet.divide(3)))
                 .and(koppen.eq(300)),
                   3001);
    
    #Cw: Pwdry<Pswet/10
    PSHwdry = ee.Image(SH.multiply(AMJJAS_prec.min()));
    PNHwdry = ee.Image(NH.multiply(ONDJFM_prec.min()));
    PSHswet = ee.Image(SH.multiply(ONDJFM_prec.max()));
    PNHswet = ee.Image(NH.multiply(AMJJAS_prec.max()));
    
    koppen=koppen.where(PSHwdry.lt(PSHswet.divide(10))
                 .or(PNHwdry.lt(PNHswet.divide(10)))
                 .and(koppen.eq(300)),
                 3002);
    
    #Cf: Psdry<40 & Psdry<Pwwet/3
    koppen=koppen.where(koppen.neq(3001)
                  .and(koppen.neq(3002))
                  .and(koppen.eq(300)),
                   3003);
    
          #Csa: Thot>=22
          koppen= koppen.where(koppen.eq(3001)
                        .and(tmax_month.max().gte(273.15+22)),
                        8);
    
          #Csb: Not (a) & Tmon10>=4 
          koppen= koppen.where(koppen.eq(3001)
                        .and(koppen.neq(8))
                        .and(tmon10.gte(4)),
                        9);
    
          #Csc: Not (a or b) & 1<=Tmon10<4
          koppen= koppen.where(koppen.eq(3001)
                        .and(koppen.neq(8))
                        .and(koppen.neq(9))
                        .and(tmon10.gte(1))
                        .and(tmon10.lt(4)),
                        10);  
    
          #Cwa: Thot>=22
          koppen= koppen.where(koppen.eq(3002)
                        .and(tmax_month.max().gte(273.15+22)),
                        11);
                        
          #Cwb: Not (a) & Tmon10>=4 
          koppen= koppen.where(koppen.eq(3002)
                        .and(koppen.neq(11))
                        .and(tmon10.gte(4)),
                        12);
                        
          #Cwc: Not (a or b) & 1<=Tmon10<4
          koppen= koppen.where(koppen.eq(3002)
                        .and(koppen.neq(11))
                        .and(koppen.neq(12))
                        .and(tmon10.gte(1))
                        .and(tmon10.lt(4)),
                        13);    
                        
          #Cfa: Thot>=22
          koppen= koppen.where(koppen.eq(3003)
                        .and(tmax_month.max().gte(273.15+22)),
                        14);
          #Cfb: Not (a) & Tmon10>=4 
          koppen= koppen.where(koppen.eq(3003)
                        .and(koppen.neq(14))
                        .and(tmon10.gte(4)),
                        15);
          #Cfc: Not (a or b) & 1<=Tmon10<4
          koppen= koppen.where(koppen.eq(3003)
                        .and(koppen.neq(14))
                        .and(koppen.neq(15))
                        .and(tmon10.gte(1)).and(tmon10.lt(4)),
                        16);                
    
    #Class D: Cold: Thot > 10°C and Tcold < 0°C
    koppen = koppen.where(ee.Image(tmax_month.max()).gte(283.15)
                   .and(ee.Image(tmax_month.min()).lte(273.15))
                   .and(koppen.neq(4))
                    .and(koppen.neq(5))
                    .and(koppen.neq(6))
                    .and(koppen.neq(7)),
                   400)
    
    #Ds: Psdry<40 & Psdry<Pwwet/3
    koppen=koppen.where(PSHsdry.lt(40)
                 .or(PNHsdry.lt(40))
                 .and(PSHsdry.lt(PSHwwet.divide(3)))
                 .or(PNHsdry.lt(PNHwwet.divide(3)))
                 .and(koppen.eq(400)),
                 4001);
    
    #Dw: Pwdry<Pswet/10
    koppen=koppen.where(PSHwdry.lt(PSHswet.divide(10))
                 .or(PNHwdry.lt(PNHswet.divide(10)))
                 .and(koppen.eq(400)),
                  4002);
    
    #Df: Psdry<40 & Psdry<Pwwet/3 
    koppen=koppen.where(koppen.neq(4001)
                  .and(koppen.neq(4002))
                  .and(koppen.eq(400)),
                   4003);
                   
          #Dsa: 
          koppen= koppen.where(koppen.eq(4001)
                        .and(tmax_month.max().gte(273.15+22)),
                        17);
    
          #Dsb: 
          koppen= koppen.where(koppen.eq(4001)
                        .and(tmon10.gte(4))
                        .and(koppen.neq(17)),
                        18);
     
         #Dsd: 
          koppen= koppen.where(koppen.eq(4001)
                        .and(koppen.neq(17))
                        .and(koppen.neq(18))
                        .and(tmax_month.min().lte(273.15-38)),
                        20);
    
          #Dsc: 
          koppen= koppen.where(koppen.eq(4001)
                        .and(koppen.neq(17))
                        .and(koppen.neq(18))
                        .and(koppen.neq(20)),
                        19);
          
          #Dwa: 
          koppen= koppen.where(koppen.eq(4002)
                        .and(tmax_month.max().gte(273.15+22)),
                        21);
          #Dwb: 
          koppen= koppen.where(koppen.eq(4002)
                        .and(tmon10.gte(4))
                        .and(koppen.neq(21)),
                        22);
          #Dwd: 
          koppen= koppen.where(koppen.eq(4002)
                        .and(koppen.neq(21))
                        .and(koppen.neq(22))
                        .and(tmax_month.min().lte(235.15)),
                        24);              
          #Dwc: 
          koppen= koppen.where(koppen.eq(4002)
                        .and(koppen.neq(21))
                        .and(koppen.neq(22))
                        .and(koppen.neq(24)),
                        23);
                
          #Dfa: 
          koppen= koppen.where(koppen.eq(4003)
                        .and(tmax_month.max().gte(273.15+22)),
                        25);
    
          #Dfb: 
          koppen= koppen.where(koppen.eq(4003)
                        .and(tmon10.gte(4))
                        .and(koppen.neq(25)),
                        26);
          #Dfc: 
          koppen= koppen.where(koppen.eq(4003)
                        .and(koppen.neq(25))
                        .and(koppen.neq(26))
                        .and(koppen.neq(28)),
                        27);
          #Dfd: 
          koppen= koppen.where(koppen.eq(4003)
                        .and(koppen.neq(25))
                        .and(koppen.neq(26))
                        .and(tmax_month.min().lte(235.15)),
                        28);
    
    #Class ET: Tundra; Thot < 10°C and Thot > 0°C
    koppen = koppen.where(ee.Image(tmax_month.max()).lte(283.15)
                   .and(ee.Image(tmax_month.max()).gte(273.15)),
                   29);
    
    #Class EF: Frost; Thot < 10°C and Thot < 0°C
    koppen = koppen.where(ee.Image(tmax_month.max()).lte(283.15)
                    .and(ee.Image(tmax_month.max()).lte(273.15)),
                    30);
    
    return koppen.mask(land)



SyntaxError: invalid syntax (1286096776.py, line 11)

In [ ]:

koppen_1950s = KOPPEN(1951, 1960, 'historical')
koppen_2090_245 = KOPPEN(2090, 2099, 'ssp245')
koppen_2090_585 = KOPPEN(2090, 2099, 'ssp585')

# Run the Koppen Geiger algorithm 

In [1]:
KoppenGeiger = kgClassification(climate_data = GDDP,first_year = 1960,last_year = 2010, model='EC-Earth3',scenario = 'historical')

NameError: name 'kgClassification' is not defined

## Reduce Regional Data

In [ ]:
#Select region
region = table.filter(ee.Filter.eq('wld_rgn','Europe')).geometry()
df = regionalStats(region)

df['Number'].sum()
df['Frequency'] = df['Number'].divide(df['Number'].sum())*100
df['Frequency'].plot(kind = 'bar')
df.to_csv('~/Data/climateClassifications/KoppenClassTest.csv')
df

In [152]:
from IPython.display import Image

holdridgePalette = ['#ea9a3d', '#447768', '#2d3aee', '#fe3a7a', '#cab2b3', '#6e219d', '#ccbddb', '#1e485e', '#7f2f4d', '#8378ee', '#83bc03', '#79fc77', '#9d4bae', '#2de630', '#b20a12', '#d518a5', '#5d59c1', '#b0d74d', '#ab2333', '#1ecbed', '#2d5c30', '#f02d58', '#b4596f', '#bd440b', '#486fa6', '#566ad0', '#386de9', '#cf71a1', '#b4b09c', '#1959e4', '#18f164', '#fdd36f', '#39fc5b', '#4ccce6', '#c26a2c', '#69cc66', '#2b84fc', '#0ea36b', '#f7e771', '#adc868', '#cf331b', '#0d7cc2', '#d4c424', '#b1df18', '#7c2093', '#0bb0db', '#3163ff', '#5cedfc', '#6b33e0', '#7b3230', '#85d6f9', '#09bbad', '#ce7d3b', '#951cf7', '#e46c84', '#088b9e', '#277b55', '#4e5c49', '#0a9fdd', '#99c553', '#6096a8', '#810953', '#b658f1', '#a6bd70', '#83a597', '#5a5c6e', '#e44a99', '#987eab', '#5a469c', '#d7b32b', '#81ccf9', '#1dfb75', '#394b9f', '#d25830', '#1bb349', '#6ec219', '#e03d9c', '#b5f9e6', '#af7bf3', '#458e65', '#8c298c', '#4273ee', '#2d42d5', '#0f6626', '#d14edb', '#fe6360', '#41f067', '#f87226', '#91177c', '#dcb2e2', '#374f4f', '#b42f90', '#989cf4', '#6c8e80', '#badd1d', '#846b5b', '#c271be', '#9f9923', '#9ed459', '#fa4f67']

# Create a URL to the styled image 
url = Holdridge.getThumbUrl({
    'min': 1, 'max': 80 , 'dimensions': 2048, 
    'palette': holdridgePalette })
print(url)

# Display the thumbnail 
print('\nPlease wait while the thumbnail loads, it may take a moment...')
Image(url=url)

https://earthengine.googleapis.com/v1/projects/ee-koppengeiger/thumbnails/57ee0b21209c1510a48bfcbde2dda1fe-8663fbd78e8782e8bbb9cc751b28c8fb:getPixels

Please wait while the thumbnail loads, it may take a moment...
